In [1]:
from train_and_test import *
from collections import defaultdict

In [2]:
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

## Testing

In [3]:
df = get_train_data("20231026", "20231129", all_stations)

2023-10-26 00:00:00
2023-10-27 00:00:00
2023-10-28 00:00:00
2023-10-29 00:00:00
2023-10-30 00:00:00
2023-10-31 00:00:00
2023-11-01 00:00:00
2023-11-02 00:00:00
2023-11-03 00:00:00
2023-11-04 00:00:00
2023-11-05 00:00:00
2023-11-06 00:00:00
2023-11-07 00:00:00
2023-11-08 00:00:00
2023-11-09 00:00:00
2023-11-10 00:00:00
2023-11-11 00:00:00
2023-11-12 00:00:00
2023-11-13 00:00:00
2023-11-14 00:00:00
2023-11-15 00:00:00
2023-11-16 00:00:00
2023-11-17 00:00:00
2023-11-18 00:00:00
2023-11-19 00:00:00
2023-11-20 00:00:00
2023-11-21 00:00:00
2023-11-22 00:00:00
2023-11-23 00:00:00
2023-11-24 00:00:00
2023-11-25 00:00:00
2023-11-26 00:00:00
2023-11-27 00:00:00
2023-11-28 00:00:00
2023-11-29 00:00:00


In [4]:
x_train, features = feature_selection(df, 0, 0, 20)

In [5]:
df_train = pd.DataFrame(np.vstack(x_train), columns=features)
df_train.head()

,station_id,date,lng,lat,hour,minute,sbi,tot,mon,tue,wed,thur,fri,sat,sun
0,500101001,2023-10-26 00:00:00,121.5436,25.02605,0,0,17,28,0,0,0,1,0,0,0
1,500101001,2023-10-26 00:20:00,121.5436,25.02605,0,20,11,28,0,0,0,1,0,0,0
2,500101001,2023-10-26 00:40:00,121.5436,25.02605,0,40,10,28,0,0,0,1,0,0,0
3,500101001,2023-10-26 01:00:00,121.5436,25.02605,1,0,9,28,0,0,0,1,0,0,0
4,500101001,2023-10-26 01:20:00,121.5436,25.02605,1,20,9,28,0,0,0,1,0,0,0


In [6]:
# add features
df_train['day'] = [d.day for d in df_train['date']]
df_train['time_str'] = [str(time).split(' ')[1] for time in df_train['date']]
df_train['isHoliday'] = [1 if df_train['sat'][i] == 1 or df_train['sun'][i] == 1 else 0 for i in range(len(df_train))]
df_train['weekday'] = [df_train['date'][i].weekday() for i in range(len(df_train))]
df_train['date_str'] = [d.strftime('%Y-%m-%d') for d in df_train['date']]

In [7]:
df_train.head()

,station_id,date,lng,lat,hour,minute,sbi,tot,mon,tue,wed,thur,fri,sat,sun,day,time_str,isHoliday,weekday,date_str
0,500101001,2023-10-26 00:00:00,121.5436,25.02605,0,0,17,28,0,0,0,1,0,0,0,26,00:00:00,0,3,2023-10-26
1,500101001,2023-10-26 00:20:00,121.5436,25.02605,0,20,11,28,0,0,0,1,0,0,0,26,00:20:00,0,3,2023-10-26
2,500101001,2023-10-26 00:40:00,121.5436,25.02605,0,40,10,28,0,0,0,1,0,0,0,26,00:40:00,0,3,2023-10-26
3,500101001,2023-10-26 01:00:00,121.5436,25.02605,1,0,9,28,0,0,0,1,0,0,0,26,01:00:00,0,3,2023-10-26
4,500101001,2023-10-26 01:20:00,121.5436,25.02605,1,20,9,28,0,0,0,1,0,0,0,26,01:20:00,0,3,2023-10-26


In [8]:
dic_tot = get_all_tot('20231002')

In [9]:
def add_FE_week_time(df_train):
    df_train['full_week_time'] = [1 if df_train['sbi'][i] > 2/3 * dic_tot[df_train['station_id'][i]] else 0 for i in range(len(df_train))]
    df_train['empty_week_time'] = [1 if df_train['sbi'][i] < 1/3 * dic_tot[df_train['station_id'][i]] else 0 for i in range(len(df_train))]

    full = df_train.groupby(['time_str', 'station_id', 'weekday'])['full_week_time'].sum().reset_index()
    empty = df_train.groupby(['time_str', 'station_id', 'weekday'])['empty_week_time'].sum().reset_index()

    df_train = pd.merge(df_train.drop(['full_week_time', 'empty_week_time'], axis=1), full, on=['time_str', 'station_id', 'weekday'])
    df_train = pd.merge(df_train, empty, on=['time_str', 'station_id', 'weekday'])
    return df_train

In [10]:
def add_FE_time(df_train, time_span, full_thrhld, emp_thrhld):
    df_train['full_time'] = [1 if df_train['sbi'][i] > 2/3 * dic_tot[df_train['station_id'][i]] else 0 for i in range(len(df_train))]
    df_train['empty_time'] = [1 if df_train['sbi'][i] < 1/3 * dic_tot[df_train['station_id'][i]] else 0 for i in range(len(df_train))]

    full = df_train.groupby(['time_str', 'station_id'])['full_time'].sum().reset_index()
    empty = df_train.groupby(['time_str', 'station_id'])['empty_time'].sum().reset_index()

    df_train = pd.merge(df_train.drop(['full_time', 'empty_time'], axis=1), full, on=['time_str', 'station_id'])
    df_train = pd.merge(df_train, empty, on=['time_str', 'station_id'])
    
    # df_train['full_time'] = df_train['full_time'] > time_span * full_thrhld
    # df_train['empty_time'] = df_train['empty_time'] > time_span * emp_thrhld
    return df_train

In [11]:
df_train = add_FE_week_time(df_train)
df_train = add_FE_time(df_train, 35, 0.7, 0.01)
df_train = df_train.sort_values(by=['station_id', 'date']).reset_index(drop=True)

In [12]:
df_train.head()

,station_id,date,lng,lat,hour,minute,sbi,tot,mon,tue,...,sun,day,time_str,isHoliday,weekday,date_str,full_week_time,empty_week_time,full_time,empty_time
0,500101001,2023-10-26 00:00:00,121.5436,25.02605,0,0,17,28,0,0,...,0,26,00:00:00,0,3,2023-10-26,1,1,9,12
1,500101001,2023-10-26 00:20:00,121.5436,25.02605,0,20,11,28,0,0,...,0,26,00:20:00,0,3,2023-10-26,1,1,9,10
2,500101001,2023-10-26 00:40:00,121.5436,25.02605,0,40,10,28,0,0,...,0,26,00:40:00,0,3,2023-10-26,1,2,5,17
3,500101001,2023-10-26 01:00:00,121.5436,25.02605,1,0,9,28,0,0,...,0,26,01:00:00,0,3,2023-10-26,1,3,4,16
4,500101001,2023-10-26 01:20:00,121.5436,25.02605,1,20,9,28,0,0,...,0,26,01:20:00,0,3,2023-10-26,1,3,5,16


In [13]:
x_public, features = get_test_data(dic_tot, "20231021", "20231024")
x_private, features = get_test_data(dic_tot, "20231218", "20231224")
df_public = pd.DataFrame(np.vstack(x_public), columns=features)
df_private = pd.DataFrame(np.vstack(x_private), columns=features)

In [14]:
# add features
def add_features(df):
    df['time_str'] = [str(time).split(' ')[1] for time in df['date']]
    df['isHoliday'] = [1 if df['sat'][i] == 1 or df['sun'][i] == 1 else 0 for i in range(len(df))]
    df['day'] = [d.day for d in df['date']]
    df['weekday'] = [df['date'][i].weekday() for i in range(len(df))]

    empty_mapping = df_train.set_index(['station_id', 'time_str', 'weekday'])['empty_week_time'].to_dict()
    full_mapping = df_train.set_index(['station_id', 'time_str', 'weekday'])['full_week_time'].to_dict()
    df['empty_week_time'] = [empty_mapping[(df['station_id'][i], df['time_str'][i], df['weekday'][i])] for i in range(len(df))]
    df['full_week_time'] = [full_mapping[(df['station_id'][i], df['time_str'][i], df['weekday'][i])] for i in range(len(df))]
    
    empty_mapping = df_train.set_index(['station_id', 'time_str'])['empty_time'].to_dict()
    full_mapping = df_train.set_index(['station_id', 'time_str'])['full_time'].to_dict()
    df['empty_time'] = [empty_mapping[(df['station_id'][i], df['time_str'][i])] for i in range(len(df))]
    df['full_time'] = [full_mapping[(df['station_id'][i], df['time_str'][i])] for i in range(len(df))]
    
    df['date_str'] = [d.strftime('%Y-%m-%d') for d in df['date']]
    # df['prcp_simple'] = [future_dic[df['date_str'][i]] for i in range(len(df))]
    return df

In [15]:
df_public = add_features(df_public)
df_private = add_features(df_private)

In [16]:
df_public.head()

,station_id,date,lng,lat,hour,minute,tot,mon,tue,wed,...,sun,time_str,isHoliday,day,weekday,empty_week_time,full_week_time,empty_time,full_time,date_str
0,500101001,2023-10-21 00:00:00,121.5436,25.02605,0,0,28,0,0,0,...,0,00:00:00,1,21,5,3,1,12,9,2023-10-21
1,500101001,2023-10-21 00:20:00,121.5436,25.02605,0,20,28,0,0,0,...,0,00:20:00,1,21,5,3,1,10,9,2023-10-21
2,500101001,2023-10-21 00:40:00,121.5436,25.02605,0,40,28,0,0,0,...,0,00:40:00,1,21,5,4,0,17,5,2023-10-21
3,500101001,2023-10-21 01:00:00,121.5436,25.02605,1,0,28,0,0,0,...,0,01:00:00,1,21,5,3,0,16,4,2023-10-21
4,500101001,2023-10-21 01:20:00,121.5436,25.02605,1,20,28,0,0,0,...,0,01:20:00,1,21,5,2,1,16,5,2023-10-21


In [17]:
df_private.head()

,station_id,date,lng,lat,hour,minute,tot,mon,tue,wed,...,sun,time_str,isHoliday,day,weekday,empty_week_time,full_week_time,empty_time,full_time,date_str
0,500101001,2023-12-18 00:00:00,121.5436,25.02605,0,0,28,1,0,0,...,0,00:00:00,0,18,0,2,1,12,9,2023-12-18
1,500101001,2023-12-18 00:20:00,121.5436,25.02605,0,20,28,1,0,0,...,0,00:20:00,0,18,0,2,0,10,9,2023-12-18
2,500101001,2023-12-18 00:40:00,121.5436,25.02605,0,40,28,1,0,0,...,0,00:40:00,0,18,0,3,0,17,5,2023-12-18
3,500101001,2023-12-18 01:00:00,121.5436,25.02605,1,0,28,1,0,0,...,0,01:00:00,0,18,0,3,0,16,4,2023-12-18
4,500101001,2023-12-18 01:20:00,121.5436,25.02605,1,20,28,1,0,0,...,0,01:20:00,0,18,0,3,0,16,5,2023-12-18


In [18]:
def model_selection(x_train, y_train, model_name):
    if model_name == 'lin_reg':
        reg = LinearRegression().fit(x_train, y_train)
        return reg
    elif model_name == 'rf':
        rf_regressor = RandomForestRegressor(n_estimators=150, random_state=42, n_jobs=-1,)
                                            #  max_depth=19, min_samples_split=8, min_samples_leaf=3)
        rf_regressor.fit(x_train, y_train)
        return rf_regressor
    elif model_name == 'svr':
        prob = problem(list(y_train), list(x_train))
        params = "-s 11"
        svr = train(prob, params)
        return svr
    elif model_name == 'adaboost':
        base_estimator = DecisionTreeRegressor(max_depth=13)
        adaboost_regressor = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=150, learning_rate=0.1, random_state=42)
        adaboost_regressor.fit(x_train, y_train)
        return adaboost_regressor
    elif model_name == 'xgboost':
        xgb = XGBRegressor(max_depth=7, n_estimators=1000, learning_rate= 0.1)
        xgb.fit(x_train, y_train)
        return xgb

In [23]:
selected_feat = ['lng', 'lat', 'hour', 'minute', 'day', 'isHoliday',  'mon', 'tue', 'wed', 'thur', 'fri', 'sat', 'sun']
model = model_selection(df_train[selected_feat].values, df_train['sbi'].values, 'rf')

In [22]:
# testing for svr
# pred_public, p_acc, _ =  predict([1 for _ in range(len(df_public))], list(df_public[selected_feat].values), model)
# pred_private, p_acc, _ =  predict([1 for _ in range(len(df_private))], list(df_private[selected_feat].values), model)

In [24]:
pred_public = testing(model, df_public[selected_feat].values, df_public['tot'])
pred_private = testing(model, df_private[selected_feat].values, df_private['tot'])

In [25]:
index1 = get_test_index("20231218", "20231224")
index2 = get_test_index("20231021", "20231024")

In [26]:
submission = pd.DataFrame([(index1 + index2 ), np.hstack((pred_private, pred_public))], index=[['id', 'sbi']]).T
submission.to_csv('../submission/phase_3/phase3_orig_data_test_weekday.csv', index=False)

In [ ]:
# submission = pd.DataFrame([(index1 + index2), np.hstack((pred_private, pred_public))], index=[['id', 'sbi']]).T
# submission.to_csv('../submission/phase1_final_submission.csv', index=False)